## Opinion Mining/ Selenium

Постепенно подбираемся к теме Sentiment Analysis.

Для начала будем считать, что у текста есть
- полярность / polarity - является ли текст эмоционально окрашенным
- тональность / sentiment - как текст оценивает некий объект по шкале -1...1 / 0...1 / 1...5 и тд

Полезные источники данных:
- сайты отзывов / сайты интернет-магазинов (товары) - Amazon ....
- сайты с медиаконтентом - Youtube, Instagram ...
- специализированные сайты - Кинопоиск, IMDB, ...

Как скачивать оттуда данные?

- requests + fake_useragent
- Selenium 
  - возможность взаимодействия с элементами (кнопки, скролл и тп)
  - обходит капчу, тк имитирует браузер


Как запустить на своем компьютере:

1. Для начала нужно скачать chromedriver (https://chromedriver.chromium.org/downloads) в рабочую папку: у вас появится архив с бинарным файлом внутри. Архив нужно будет распаковать, а путь к папке добавить в path. Можно пользоваться драйверами различных браузеров, здесь я оставляю ссылки на версию google chrome.
2. В начале скрипта мы создаем какую-либо переменную, в которую помещаем наш браузер.
Пр.: driver = webdriver.Chrome(<полный путь к бинарному файлу>)
3. Открыть страницу можно с помощью функции get. 
Прокрутить страницу можно с помощью кода внизу.
4. Находить различные элементы страницы можно с помощью ряда функций find_elements_by_<...> (см. документацию).
Чтобы понять, как найти какой-либо элемент на странице, нужно в веб-инспекторе в браузере посмотреть путь элемента. В гугл-хроме это можно сделать, например, так: Посмотреть -> Разработчикам -> Проверить элементы -> Навести стрелкой на нужный элемент.
5. Как только мы нашли нужный элемент, можно на него кликнуть функцией .click() или посмотреть текст функцией .text().
6. В конце программы нужно  закрыть, если открывалли как сверху и, конечно, как -то сохранить ваши данные

Подробнее: https://towardsdatascience.com/how-to-scrape-youtube-comments-with-python-61ff197115d4

In [ ]:
!pip3 install selenium

## Данные

Возьмем противоречивое видео с ютьюба и попробуем проанализировать комментарии к нему
 https://www.youtube.com/watch?v=kuhhT_cBtFU&t=2s

## Тетрадка для подготовки данных 
[в Colab](https://colab.research.google.com/drive/1IFyhNz3UCvIEgHjskLNIgD0UddvxCkPw?usp=sharing)

In [ ]:
!pip install textblob stanza spacy

In [ ]:
# Загружаем модель для SpaCy
!python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import re, string
import nltk
from nltk.corpus import stopwords
import numpy as np
import stanza
import spacy
from textblob import TextBlob

spacy_nlp = spacy.load('en_core_web_sm')

stanza.download("en")
stanza_nlp = stanza.Pipeline("en", processors="tokenize,lemma,pos,sentiment")

nltk.download('stopwords')
nltk.download('punkt')
sw = stopwords.words('english')

# Очистка текстов
def clean_text(text):
    text = text.lower()
    text = re.sub('@', '', text)
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"[^a-zA-Z ]+", "", text)
    return text

In [ ]:
def lemmatize(text):  # spacy
    doc = spacy_nlp(text)
    return [token.lemma_ for token in doc]

def lemmatize_stanza(text):  #stanza
    doc = stanza_nlp(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]

In [ ]:
def remove_stopwords(tokens):
    return [t for t in tokens if t not in sw]

In [ ]:
def pipelinize(texts):
    return spacy_nlp.pipe(texts, disable=["parser", "ner"])

In [ ]:
pd.set_option('display.max_colwidth', 1000)

In [ ]:
a = pd.read_csv('youtube_comments.txt', sep='\t')

In [ ]:
a['clean_text'] = a['text'].apply(lambda x: clean_text(x))

In [ ]:
a.sample(frac=0.01)

In [ ]:
sample_texts = a.sample(frac=0.1)

In [ ]:
len(sample_texts)

In [ ]:
lemmas_comments = []
for doc in pipelinize(sample_texts['clean_text'].to_list()):
    lemmas_comments.append([token.lemma_ for token in doc])
sample_texts['lem'] = lemmas_comments

In [ ]:
sample_texts['best'] = sample_texts['lem'].apply(lambda x: remove_stopwords(x))

In [ ]:
sample_texts.sample(frac=0.01)

## Предварительный статистический анализ

Как предварительно проанализировать данные, чтобы понимать, куда копать?

### Посмотрим на распределение частот лемм

In [ ]:
#Your code here - freq

### Посмотрим на частоты биграмм и триграмм:
- `nltk.bigrams`

In [ ]:
#bigram freq

In [ ]:
#trigram freq

### А если убрать слишком короткие комментарии?

## TextBlob

Простая библиотека, которая совмещает статистические методы и regexp-паттерны.

The sentiment property returns a namedtuple of the form `Sentiment(polarity, subjectivity)`. 

The polarity score is a float within the range [-1.0, 1.0]. 

The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [ ]:
blob = TextBlob(sample_texts['text'][311276])
blob.sentiment

In [ ]:
blob.sentiment_assessments

In [ ]:
def comment_sentiment(comment):
    blob = TextBlob(comment)
    return np.mean([s.sentiment.polarity for s in blob.sentences])


In [ ]:
sample_texts['sent'] = sample_texts['text'].apply(lambda x: comment_sentiment(x))

In [ ]:
sample_texts

In [ ]:
sample_texts['sent'].plot.hist(color='salmon', title='Comments Polarity')

In [ ]:
sample_texts[sample_texts['sent']<-0.5]

## Stanza Sentiment Analysis

In [ ]:
stanza_sentiment = ['negative', 'neutral', 'positive']
for sentence in stanza_nlp(sample_texts['clean_text'][294519]).sentences:
    print(stanza_sentiment[sentence.sentiment])